# Tutorial on AixCaliBuHA

*Before going any further, please note that you first should read the tutorial on `ebcpy` ([Link](https://git.rwth-aachen.de/EBC/EBC_all/Python/ebcpy)). The basic data types are explained there. Also, an example for the simulation API is made hinting on AixCaliBuHa and why you may need it.*

### What is AixCaliBuHA?

**Aix** (from French Aix-la-Chapelle) **Cali**bration for **Bu**ilding and **H**V**A**C Systems

This framework attempts to make the process of calibrating models used in building
and HVAC systems easier. Different sub packages help with the underlying tasks of:

- Performing a **Sensitivity Analysis** to discover tuner parameters for the calibration
- **Calibration** of given model based on the tuner parameters, the calibration classes and specified goals to evaluate the objective function of the underlying optimization

### Why Calibration

When modelling Building and HVAC-Systems, one wants to make valid statements about the real system. A model which represents some real-world process in a sufficient manner (e.g. in terms of supply temperature) may be useful to makes such statements. Model parameters have to be tuned so that some simulation output matches the output in the real world (e.g. some measurement). 
- **Which parameters to tune?** You as a modeler may already know important parameters that influence your model regarding a targeted output. However, performing a **sensitivity analysis** may be helpful to quantify which parameters are important.
- **How to tune?** Manual tuning is most often still state of the art. However it is very inefficient. Therefore, the appraoch in this framework is the combination of mathematical **optimization** for calcuation of the next **tuner parameter** value and Simulation APIs for **automation of the optimization**.

### Content of this Tutorial:
 1. [The Basics: Tuner Paramateres, Goals and Calibration-Classes](#basics)
 2. [Sensitivity Analysis: Get to know your model parameters](#sensanalysis)
 3. [Modelica-Calibration: Getting started on calibration](#single_cal)
 4. [Advanced Calibration: Multiple Classes, kwargs, solvers and best practices](#adv_cal)
  1. [Multiple-Classes Calibration](#mult_cal)
  2. [kwargs - Settings of the Calibrator](#kwargs_cal)
  3. [Solver Options](#kwargs_solver)
  4. [Best Practices](#best_practices)
  5. [Visualization: The different plots explained](#visual)

<a id='basics'></a>
## The Basics: Tuner Paramaters, Goals and Calibration Classes

Partly introduced in `ebcpy`, we will briefly explain the underlying `data_types`.

### Tuner Parameters:

All model parameters used either in a Sensitivity Analysis, an Optimization or a Calibration are at some point tuned, thus are tuner paramteres.
Basically, a tuner parameter has a name (string), an initial value and minimal/maximal values (floats). For efficient optimization, the values are internally normalized to the range 0..1 (if min/max values (bounds) are given. This way different units (e.g. Temperature / K and Pressure / Pa) behave the same during optimization. Some solvers require boundaries some not. Initial values are not required for some global or stochastic solvers. 

In [1]:
from aixcalibuha import TunerParas

tuner_paras = TunerParas(
        # A list with the names of the parameters in the model.
        names=["HeatExchanger.G", "HeatExchanger.C"], 
        # List with initial values as floats
        initial_values=[25, 3000],
        # List with tuples. First item is minimal, second item maximal value
        bounds=[(0, 300), (0, 10000)])

# Lets look at the object (a DataFrame is internally used)
print(tuner_paras)

                 initial_value  min    max  scale
names                                            
HeatExchanger.G             25    0    300    300
HeatExchanger.C           3000    0  10000  10000


In [2]:
# Most functions (scale, descale, etc.) are used internally for automatic calibration or similiar. 
# You may find useful:
tuner_paras.set_value("HeatExchanger.G", "max", 400)
tuner_paras.remove_names(["HeatExchanger.C"])
print(tuner_paras)

                 initial_value  min  max  scale
names                                          
HeatExchanger.G             25    0  400    400


### Goals:

Goals are used to evaluate the difference between measured and simulated data. You may want to calibrate your model based on multiple values, e.g. power consumption, supply temperature etc.
As mentioned in `ebcpy`, we use our own **multi index `DataFrame`**. Here with the row names `Variables` and `Tags`. Looking at `Goals`, we therefore have the following structure:

In [3]:
from ebcpy import data_types, preprocessing
from aixcalibuha import Goals
import os

# First we load our measurement data as time series data
path = os.path.join(os.getcwd(), "data", "ref_result.hdf")
mtd = data_types.TimeSeriesData(path, key="test")
#mtd = preprocessing.build_average_on_duplicate_rows(mtd)
print(mtd)
# I refer to the docstring of the class for more information. If you have further questions, please raise an Issue.
print(Goals.__doc__)

# Recall from the doc-strings that the dict has the following structure:
# variable_names = {VARIABLE_NAME: [MEASUREMENT_NAME, SIMULATION_NAME]}
variable_names = {"T_heater": ["measured_T_heater", "heater.heatPorts[1].T"],
                  "T_heater_1": ["measured_T_heater_1", "heater1.heatPorts[1].T"]}

goals = Goals(meas_target_data=mtd,
              variable_names=variable_names,
              # Use weightings if one of the goals is more relevant than another. A weighted sum is applied,
              # for example with [0.9, 0.1] the total objective will be = 0.9 * objective_goal_1 + 0.1 * objective_goal_2  
              weightings=[0.3, 0.7])

print(goals)

Variables           measured_T_heater measured_T_heater_1
Tags                              raw                 raw
Time                                                     
2020-03-01 15:10:49        313.165863          293.173126
2020-03-01 15:10:50        312.090271          293.233002
2020-03-01 15:10:51        312.090027          293.385925
2020-03-01 15:10:52        312.109436          293.589233
2020-03-01 15:10:53        312.130341          293.819611
...                               ...                 ...
2020-03-01 16:10:45        312.790192          302.844788
2020-03-01 16:10:46        312.790192          302.844788
2020-03-01 16:10:47        312.790192          302.844788
2020-03-01 16:10:48        312.790192          302.844788
2020-03-01 16:10:49        312.790192          302.844788

[3601 rows x 2 columns]

    Class for one or multiple goals. Used to evaluate the
    difference between current simulation and measured data

    :param (ebcpy.data_types.TimeSeriesData

Now let's assume we've run a simulation and want to add the result data to our Goals object. (This is done automatically in the Calibration process.) First we load the simulation data and analyze it:

In [4]:
path = os.path.join(os.getcwd(), "data", "simTargetData.mat")
# The class data_types.TimeSeriesData can handle both *.hdf and *.mat files. 
# Latter one are the default result file format from Modelica simualtions.
std = data_types.TimeSeriesData(path)
print(std)

Variables heater.heatPorts[1].T heater1.heatPorts[1].T
Tags                        raw                    raw
Time                                                  
0.0                  313.165863             293.173126
1.0                  310.787750             293.352448
2.0                  310.796753             293.719055
3.0                  310.870331             294.141754
4.0                  310.936859             294.564545
...                         ...                    ...
596.0                307.777649             307.777649
597.0                307.777649             307.777649
598.0                307.777649             307.777649
599.0                307.777649             307.777649
600.0                307.777649             307.777649

[601 rows x 2 columns]


Looking at the data, we may already expect some error. Why? Because our sim-target-data is using some `float`-based index but our measurement data is using `DatetimeIndex` as an index. We can solve this issue in two ways: Either convert `std` to a `DatetimeIndex` or `mtd` to a `float`-based index. Which way to choose? We agreed on the latter one, mainly because of efficiency.  
`set_sim_target_data` is called every iteration, therefore the conversion would take place every iteration. Iteration here means that during the subsequent calibration process the simulation is called multiple times and this always calls afterwards the `set_sim_target_data` function. Thus, processing the `std` with the default `float`-based index saves the operation of converting the index type. In contrast, `mtd` index needs to be converted only once! To display your result in the end you may use the original index again, but for the calibration, the index of `std` is used. For the conversion, we offer a preprocessing function in `ebcpy`:

In [5]:
try:
    goals.set_sim_target_data(sim_target_data=std)
except IndexError as e:
    print("IndexError:", e)
    
path = os.path.join(os.getcwd(), "data", "ref_result.hdf")
mtd = data_types.TimeSeriesData(path, key="test")
mtd.to_float_index()
# Now we may construct our goals and set the simulation-target-data:
goals = Goals(meas_target_data=mtd,
              variable_names=variable_names,
              weightings=[0.3, 0.7])
goals.set_sim_target_data(std)
# Let's look at the data:
print(' ') # Just add an empty line
print(goals)

IndexError: Given sim_target_data is using Float64Index as an index, but the reference results (measured-data) was declared using the DatetimeIndex. Convert your measured-data index to solve this error.
 
Variables    T_heater              T_heater_1            
Tags             meas         sim        meas         sim
Time                                                     
0.0        313.165863  313.165863  293.173126  293.173126
1.0        312.090271  310.787750  293.233002  293.352448
2.0        312.090027  310.796753  293.385925  293.719055
3.0        312.109436  310.870331  293.589233  294.141754
4.0        312.130341  310.936859  293.819611  294.564545
...               ...         ...         ...         ...
3596.0     312.790192         NaN  302.844788         NaN
3597.0     312.790192         NaN  302.844788         NaN
3598.0     312.790192         NaN  302.844788         NaN
3599.0     312.790192         NaN  302.844788         NaN
3600.0     312.790192         NaN  302.84

We can clearly see how the sim-target-data was added. In a calibration, each variable is internally evaluated using some statistical measure that you provide. But more on that in the part for calibration. Don't worry about the `NaN` values, we will handle them using the python-class `CalibrationClass`. They occur due to the fact that our measurement is 1 hour long, but the example simulation data only 10 minutes. Internally, `set_relevant_time_intervals` is used to just include the data in a specific interval (the interval is provided in the `CalibrationClass` object). 
You may play with the `statistical_measure`. The verbose option is used for the logger to better visualize how the weightings affect the end result and which goal is performing well and which not so much.  

In [6]:
goals.set_relevant_time_intervals([(0, 600)])
print(goals)
print(goals.eval_difference(statistical_measure="NRMSE", verbose=True))
# Here is the list of all avaliable statistical measures:
from ebcpy.utils.statistics_analyzer import StatisticsAnalyzer
help(StatisticsAnalyzer)

Variables    T_heater              T_heater_1            
Tags             meas         sim        meas         sim
Time                                                     
0.0        313.165863  313.165863  293.173126  293.173126
1.0        312.090271  310.787750  293.233002  293.352448
2.0        312.090027  310.796753  293.385925  293.719055
3.0        312.109436  310.870331  293.589233  294.141754
4.0        312.130341  310.936859  293.819611  294.564545
...               ...         ...         ...         ...
596.0      312.790192  307.777649  302.844849  307.777649
597.0      312.790192  307.777649  302.844849  307.777649
598.0      312.790192  307.777649  302.844849  307.777649
599.0      312.790192  307.777649  302.844849  307.777649
600.0      312.790192  307.777649  302.844849  307.777649

[601 rows x 4 columns]
(1.0444056379002862, {0.3: 2.6869453543928863, 0.7: 0.3404600451177434})
Help on class StatisticsAnalyzer in module ebcpy.utils.statistics_analyzer:

class Statisti

### Calibration Classes:
Last but not least we need an object to define in what time interval we want to calibrate our simulation model.
We call this time interval **Calibration Class**, not to be confused with the **python class**. A `CalibrationClass` contains everything we need to run the calibration.  
We need to ask ourselves: **What type of `CalibrationClass` are we talking about?**  
Depending on the time interval, different tuner parameters and goals are relevant to a calibration. If a device is turned on (e.g. `name="Device On"`), the power consumption may be a goal. If it is turned off, the temperature losses may become important and, thus, parameters like heat conductivity should represent a tuner parameter.  
We therefore need: `name`, `goals`, `tuner_parameters`. Additionally we need a time interval in which we want to compare our data `start_time` and `stop_time`. 

**To detect such classes, the EBC offers `EnSTats` ([Link](https://git.rwth-aachen.de/EBC/EBC_all/Optimization-and-Calibration/enstats)), a python-library to classify and cluster time-series-data. The output may be used for Calibration.**

Let's declare our CalibrationClass:

In [7]:
from aixcalibuha import CalibrationClass

cal_class = CalibrationClass(name="Device On",  # Said name of the class
                             start_time=0, 
                             stop_time=600,
                             goals=goals,
                             tuner_paras=tuner_paras,
                             # The relevant intervals are mainly useful for calibration of multiple classes. 
                             # If you specify like below, the simulation will run from 0 to 600s, but when calling
                             # eval_difference, only the intervals 0-100 and 500-600 are relevant. 
                             relevant_intervals=[(0, 100),(500, 600)])

You may open and run the file under `\examples\cal_classes_example.py`. The example already uses a `list` to store multiple classes. For a standard calibration, you most likely will encounter multiple `CalibrationClass`es. Therefore we normally use a list of `CalibrationClass` objects. The order is not important. Internally, classes with the same names are converted into one `CalibrationClass`. Example (from the function `merge_calibration_classes`):

In [8]:
from aixcalibuha import merge_calibration_classes
cal_classes = [CalibrationClass("on", 0, 100),
               CalibrationClass("off", 100, 200),
               CalibrationClass("on", 200, 300)]
merged_classes = merge_calibration_classes(cal_classes)
# Is equal to:
merged_classes = [CalibrationClass("on", 0, 300,
                                    relevant_intervals=[(0,100), (200,300)]),
                  CalibrationClass("off", 100, 200)]

# Test:
print(merged_classes[0].relevant_intervals)

[(0, 100), (200, 300)]


<a id='sensanalysis'></a>
## Sensitivity analysis: Get to know your model parameters

So far we've mostly talked about calibration. An important step towards a succesful calibration is a sensititity analysis. Be it using our tool or using other applications (Dymola offers some options - see sweep parameters), you have to know which model parameters affect which output value of your model. Not only for calibration, but also for later application of the model for studies this is vital.

We won't go into much detail about the theory behind a sensitivity analysis. Taken from [Wikipedia](https://en.wikipedia.org/wiki/Sensitivity_analysis):
> "Sensitivity analysis is the study of how the uncertainty in the output of a mathematical model or system (numerical or otherwise) can be divided and allocated to different sources of uncertainty in its inputs"

### What do we need to perform a sensitivity analysis?

Looking at the definition, we will need a **model**, **output values** and **input values**.
The model is provided using the `simulation_api` of `ebcpy`. Output values are in our case our `goals`, because these outputs are relevant for our calibration. Input values are the tuner parameters (`tuner_paras`), as we want to know the uncertainty of each parameter on our goals.  
Additionally, we need to specify which analysis we want to perform. We provide an increasing set of methods, look at the docstrings to know which methods are supported.

For more on the methods, check: [morris](https://salib.readthedocs.io/en/latest/api.html#method-of-morris) and/or [sobol](https://salib.readthedocs.io/en/latest/api.html#sobol-sensitivity-analysis)

### How do we implement it?

Adapted from the file `sen_analysis_example.py` in the examples folder.

**Note:** To limit the execution time of this code, we use `num_samples=2`. The results are obviously bad, but this is just the tutorial to get you familiar with syntax and output format etc. 

**IMPORTANT:** Dymola needs to be installed, because it is called via its `python_interface` in the next code block.

In [15]:
from ebcpy.examples import dymola_api_example
from aixcalibuha.examples import cal_classes_example
from aixcalibuha.sensanalyzer import SobolAnalyzer, MorrisAnalyzer
import os
import pandas as pd

# %% Parameters for sen-analysis:
# Used to call `goals.eval_difference`.
statistical_measure = "RMSE"
# Check out the ebcpy tutorial for an introduction to the simulation API.
cd = os.path.normpath(os.path.join(os.getcwd(), "testzone"))
sim_api = dymola_api_example.setup_dymola_api(cd=cd)
# Pick out calibration classes from the example file
cal_classes = cal_classes_example.setup_calibration_classes()[0:2]

# %% Sensitivity analysis:
# So far the methods Morris and Sobol are available options. We refer to SALib's documentation on these methods.
# For the present case, 8 samples are generated (parameters(3) + 1) * num_samples(2) = 8

sen_analyzer = MorrisAnalyzer(cd=sim_api.cd,  # cd is used for logging
                              sim_api=sim_api,
                              num_samples=2,
                              statistical_measure=statistical_measure)

# Evaluate and quantify which tuner parameter has which influence on which class
sen_result = sen_analyzer.run(calibration_classes=cal_classes)

for result in sen_result:
    print(pd.DataFrame(result))
# Close Dymola
sim_api.close()

04.05.2021-14:41:43 INFO DymolaAPI: -------------------------Initializing class DymolaAPI-------------------------
04.05.2021-14:41:43 INFO DymolaAPI: -------------------------Initializing class DymolaAPI-------------------------
04.05.2021-14:41:43 INFO DymolaAPI: -------------------------Initializing class DymolaAPI-------------------------
04.05.2021-14:41:43 INFO DymolaAPI: -------------------------Initializing class DymolaAPI-------------------------
04.05.2021-14:41:46 INFO DymolaAPI: Loading Model Modelica
04.05.2021-14:41:46 INFO DymolaAPI: Loading Model Modelica
04.05.2021-14:41:46 INFO DymolaAPI: Loading Model Modelica
04.05.2021-14:41:46 INFO DymolaAPI: Loading Model Modelica
04.05.2021-14:41:46 INFO DymolaAPI: Loaded modules
04.05.2021-14:41:46 INFO DymolaAPI: Loaded modules
04.05.2021-14:41:46 INFO DymolaAPI: Loaded modules
04.05.2021-14:41:46 INFO DymolaAPI: Loaded modules
04.05.2021-14:41:46 INFO MorrisAnalyzer: Start sensitivity analysis of class: Heat up, Time-Interval

                                             mu_star  \
0  [0.1491764446140984, 1.2575638158693754, 2.712...   

                                        mu_star_conf  \
0  [0.04916387106912529, 0.2149542846387027, 0.51...   

                       names  \
0  [C, m_flow_2, heatConv_a]   

                                               sigma  \
0  [0.04999672514349026, 1.7784638039521323, 0.53...   

                                                  mu  duration[s]  
0  [-0.1491764446140984, 0.15932462172930995, -2....    34.516285  
                                                   0
0  <aixcalibuha.CalibrationClass object at 0x0000...


04.05.2021-14:42:22 INFO DymolaAPI: Successfully closed Dymola
04.05.2021-14:42:22 INFO DymolaAPI: Successfully closed Dymola
04.05.2021-14:42:22 INFO DymolaAPI: Successfully closed Dymola
04.05.2021-14:42:22 INFO DymolaAPI: Successfully closed Dymola


What parameters do we now extract? This is up to you. We implemented a small function `automatic_select`, where we receive all tuner parameters below a certain `threshold` for a given `key`. Be cautious using this function. First look at the results yourself and see which threshold is a good one.

In [ ]:
cal_classes = sen_analyzer.automatic_select(calibration_classes=[cal_classes[0]],
                                            result=sen_result,
                                            threshold=1,
                                            key="mu_star")

<a id='single_cal'></a>
## Modelica calibration: Getting started on calibration

The simplest type of calibration in Modelica is the single-class calibration. The `Calibrator` class of `aixcalibuha` inherites from the `Optimizer` class in `ebcpy`. This is due to the fact that a calibration is an optimization. 

At this point you should already know the things you need for a calibration: `tuner_parameters`, `goals` and at least one `CalibrationClass`. Furthermore, a model is necessary (hence `simulation_api`), and a `statistical_measure` to evaluate the difference between measured and simulated data. Further keyword arguments (`kwargs`) may help with a successful calibration. The next section (Advanced calibration) goes into more detail on that. Read the docstring of the classes to learn more about avaiable options. If you have questions, as always, please raise an issue.

The following code is based on the example file: `aixcalibuha\examples\calibration_example.py`

**Note:** To limit the execution time of this code, we use the solver/method/framework-specific kwargs `maxiter=0` and `popsize=1`. This limits the number of simulations drastically (to around 9 in the present case). The results are obviously bad, but this is just the tutorial to get you familiar with syntax and output format etc. 

In [ ]:
import os
from ebcpy.examples import dymola_api_example
from aixcalibuha.calibration import modelica
from aixcalibuha.examples import cal_classes_example

# Equivalent to the setup of the sensitivity analyzer:
statistical_measure = "RMSE"
cd = os.path.normpath(os.path.join(os.getcwd(), "testzone"))
simulation_api = dymola_api_example.setup_dymola_api(cd=cd)
cal_classes = cal_classes_example.setup_calibration_classes()
# For the single-class, we only need one. Let's take the first one:
cal_class = cal_classes[0]
# Choose the framework (required): See the docstrings of ebcpy.optimization.optimize for available options.
framework="scipy_differential_evolution"
# Choose solver / method (required only for some frameworks). For instance, the framework 'dlib_minimize' would not need a method.
method="best1bin"
# More on kwargs later. This is just to limit runtime here
# Useable kwargs are highly dependent of selected framework and solver method!
kwargs_calibrator = {"maxiter":0, "popsize": 1}
# The interactive plotting has some bugs in jupyter.
kwargs_calibrator.update({"show_plot" : False})

# Setup the class
single_class_cal = modelica.ModelicaCalibrator(cd=simulation_api.cd,
                                               sim_api=simulation_api,
                                               statistical_measure=statistical_measure,
                                               calibration_class=cal_class,
                                               **kwargs_calibrator)
# Start the calibration process
single_class_cal.calibrate(framework=framework, method=method)
# Close dymola
simulation_api.close()

<a id='adv_cal'></a>
## Advanced calibration: Multiple classes, kwargs, solvers and best practices

You should now know the basics of this framework. To get into a little bit more detail, we've prepared different sections in the following. Check out any one you find interesting and want to learn more about:

1. **Multiple classes**: What strategies exists for the calibration?
2. **kwargs**: How can I change the settings of the calibrator?
3. **Solver options**: How can I tune the optimizer to enhance the results?
4. **Best practices**: Some general advices for getting good results

<a id='mult_cal'></a>
### 1. Multiple classes calibration

Calibrating one `CalibrationClass` is a straight foreward task. You have one set of tuner parameters and one time interval. When aiming to calibrate multiple classes, different possible strategies arise to fulfill the goal of calibration. We've come up with different approaches and narrowed down to two options:

<img src="tutorial/multiple_class_calibration.png">

Looking at the code, these two options are implemented as follows:

In [ ]:
import os
from ebcpy.examples import dymola_api_example
from aixcalibuha.calibration import modelica
from aixcalibuha.examples import cal_classes_example

# Equivalent to the setup of the single-class-calibration:
statistical_measure = "RMSE"
cd = os.path.normpath(os.path.join(os.getcwd(), "testzone"))
simulation_api = dymola_api_example.setup_dymola_api(cd=cd)
cal_classes = cal_classes_example.setup_calibration_classes()
framework="scipy_differential_evolution"
method="best1bin"
# Choose options for multiple-class-calibration:
start_time_method="timedelta"  # Or "fixstart"
# This parameter is coupled to the parameters fix_start_time and timedelta
# For timedelta you have to pass a timedelta (e.g. timedelta=10 -> timedelta of 10 s), 
# For fixstart you have to pass a fix_start_time (e.g. fix_start_time=0 s -> start_time=0 for all classes)
# More on kwargs later. This is just to limit runtime here
kwargs_calibrator = {"maxiter":0, "popsize":1}
# The interactive plotting has some bugs in jupyter.
kwargs_calibrator.update({"show_plot" : False})

# Setup the class
multiple_class_cal = modelica.MultipleClassCalibrator(
                                    cd=simulation_api.cd,
                                    sim_api=simulation_api,
                                    statistical_measure=statistical_measure,
                                    calibration_classes=cal_classes,
                                    start_time_method=start_time_method,
                                    timedelta=0,
                                    **kwargs_calibrator)
# Start the calibration process
multiple_class_cal.calibrate(framework=framework, method=method)
# Close dymola
simulation_api.close()

We could now open the files under `aixcalibuha/examples/testzone` and see the results of the calibration. However, as we only used a limit number of function evaluations, we refer to section "5. Visualization: The different plots explained".

<a id='kwargs_cal'></a>
### 2. kwargs - Settings of the calibrator:

Different keyword arguments act as settings for the calibration. Although solver-specific attributes are also set using keyword arguments, we only introduce the calibration-specific keyword arguments here. See the section on solver-specific kwargs for more on that. 

All keyword arguments are explained in the docs. We therefore can either open the documentation ([This direct link](https://ebc.pages.rwth-aachen.de/EBC_all/Optimization-and-Calibration/AixCaliBuHA/master/docs/) is referenced in the [Documentation section of the Readme.md](https://git.rwth-aachen.de/EBC/EBC_all/Optimization-and-Calibration/AixCaliBuHA#documentation)) or look at the docstring:

In [ ]:
from aixcalibuha.calibration import modelica
print(modelica.ModelicaCalibrator.__doc__)

In [ ]:
# Let's use a dict to easily setup our keyword arguments:
# Specify values for keyword arguments to customize the Calibration process for a single-class-calibration
kwargs_calibrator = {"timedelta": 0,
                     "save_files": False,
                     "verbose_logging": True,
                     "show_plot": True,
                     "create_tsd_plot": True,
                     "save_tsd_plot": False}

For the multiple class calibration, additional kwargs exist:

In [ ]:
print(modelica.MultipleClassCalibrator.__doc__)

In [ ]:
# If we use multiple classes, we may update our original dict:
kwargs_multiple_classes = {"merge_multiple_classes": True,
                           "fix_start_time": 0,
                           "timedelta": 0}
if len(cal_classes) > 1:
    kwargs_calibrator.update(kwargs_multiple_classes)

<a id='kwargs_solver'></a>
### 3. Solver options:

Introduced in the `ebcpy`-Tutorial, the following options are not directly part of AixCaliBuHA. However, some are quite useful for the use case of calibration and we will, therefore, highlight them shortly. Note that we refer to the original documentation of each framework/method/solver for a more detailed explanation of each parameter.

**Note** All values below are the default values. Finding good values is up to the user. Not all keywords are necessary for all methods. 

Info on:
- [scipy_differential_evolution](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.differential_evolution.html)
- [scipy_minimize](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.minimize.html)
- [dlib_minimize](http://dlib.net/python/index.html#dlib.find_min_global)

In [ ]:
# Specify solver-specific keyword-arguments depending on the solver and method you will use
# scipy differential evolution. 
kwargs_scipy_dif_evo = {"maxiter": 1000,
                        "popsize": 15,
                        "mutation": (0.5, 1),
                        "recombination": 0.7,
                        "seed": None,
                        "polish": True,
                        "init": 'latinhypercube',
                        "atol": 0}
# Dlib: num_function_calls is the maximal number of iterations. 
# Something like 400 maybe is a good starting point
kwargs_dlib_min = {"num_function_calls": int(1e9),
                   "solver_epsilon":0}

# Scipy-minimize:
kwargs_scipy_min = {"tol": None,
                    "options": None,
                    "constraints": None,
                    "jac": None,
                    "hess": None,
                    "hessp": None}

# Merge the dictionaries into one.
# If you change the solver, also change the solver-kwargs-dict in the line below. 
# In the example, a simple if-case handels this automatically
kwargs_calibrator.update(kwargs_dlib_min)

<a id='best_practices'></a>
### 4. Best practices

When performing an optimization, one should know the [**no free lunch theorem**](https://en.wikipedia.org/wiki/No_free_lunch_in_search_and_optimization). Therefore, we cannot tell you which solver is the best and which settings to choose. However, as we are limited to Building Energy Systems, the following best practices may be a good starting point for you. They are purely based on experience in calibration and are neither empirically grounded or mathematically proven. Nevertheless, here we go:

- **Contribute to this list**: If you learn/know something useful about calibration which should be in this list, add it!
- **Contribute to AixCaliBuHA**: A lot of people have the same problems you have/had. Although it consumes more time than writing a quick&dirty script on your own: Help to expand and improve this framework by contributing. Not only others will thank you in the future.
- **Use gradient-free solvers for Modelica**: Calibrating a simulation model is a black box for the solver. Therefore, gradient-free methods (`dlib_minimize`, `scipy_differential_evolution`) yield better results than most methods of e.g. `scipy_minimize`. The latter require the gradient of a function.
- **Check your initial values**: Using `AixCaliBuHA`, we provide visualization to help you identify possible points of improvment. One is the initialization. If the difference is big, you may either use `timedelta` for a steady-state intial condition OR use explicit start values which you write in the Modelica model. You may alter these start values through the `dymola_api`.
- **Test different solver (and options)**: This is maybe time consuming. But try out different options and see what works best for you. If you read about a solver which works well for you, raise an isse: We can include it into the framework pretty fast.
- **Pre-process your data**: Noisy input data will most certaintly yield bad results.
- **Never ever copy results blindly**: The two points below further explain why. We are engineers and should always question the solution the calibration/simulation yields.
- **Don't overfit**: Especially using stocahstical methods, you will find (after 1000 of iterations) pretty good paramteres. However keep in mind that these parameters also have to be validated
- **Question unphysical parameteres**: You modelled a physical system but some tuned parameter makes no sense? Boundaries might have been set to broad. Check if the model or the measurement is physically coherent. If not, adapt it.

<a id='visual'></a>
### 5. Visualization: The different plots explained

We provide different plots to make the process of calibration clearer to you. We will go into detail on the different plots, what they tell you and how you can enable/disable them. We refer the plot names with the file names they get.

---

#### objective_plot:
<img src="tutorial/objective_plot.svg">

**What do we see?** The solver in use was "scipy_differential_evolution" using the "best1bin" method. After around 200 iterations, the solver begins to converge. The last 150 itertions don't yield a far better solution, it is ok to stop the calibration here. You can do this using a `KeyboardInterrupt` / `STRG + C`.

**How can we enable/disable the plot?** Using the `show_plot=True` keyword argument (default is `True`)

---

#### tuner_parameter_plot:
<img src="tutorial/tuner_parameter_plot.svg">

**What do we see?** The variation of values of the tuner parameters together with their specified boundaries (red lines). The tuner parameters vary significantly in the first 200 iterations. At convergence the values obviously also converge.

**How can we enable/disable the plot?** Using the `show_plot=True` keyword argument (default is `True`)

---

#### tsd_plot: Created for two different classes - "stationary" and "Heat up"
<img src="tutorial/tsd_plot_heat_up.svg">
<img src="tutorial/tsd_plot_stationary.svg">

**What do we see?** The measured and simulated trajectories of our selected goals. The grey part is not used for the evaluation of the objective function. As these values are `NaN`, matplotlib may interpolate linearly between the points, so don't worry if the trajectory is not logical in the grey area. Note that the inital values for the class "stationary" are not matching the initial values of the measured data. Even if the parameters are set properly, the objective would yield a bad result. In this case you have to adapt the inital values of your model directly in the Modelica code (see section "Best practices").

**How can we enable/disable the plot?** Using the `create_tsd_plot=True` keyword argument for showing it each iteration, the  `save_tsd_plot=True` for saving each of these plots. (Default is `True` and `False`, respectivly.)

---

#### tuner_parameter_intersection_plot:
<img src="tutorial/tuner_parameter_intersection_plot.svg">

**What do we see?** This plot is generated if you calibrate multiple classes **AND** different classes pyrtially have the same tuner parameters (an intersection of `tuner_paras`). In this case multiple "best" values arise for one tuner parameter. The plot shows the distribution of the tuner-parameters if an intersection is present. You will also be notified in the log file. In the case this plot appears, you have to decide which value to choose. If they differ greatly, you may want to either perform a sensitivity analysis to check which parameter has the biggest impact OR re-evaluate your modelling decisions. 

**How can we enable/disable the plot?** Using the `show_plot=True` keyword argument (default is `True`)
